In [1]:
%run input/Format.ipynb
import ROOT as root
from array import array
from math import sin
from ROOT import gRandom
from ROOT import TFile
from ROOT import TPad, TLegend
root.gErrorIgnoreLevel = root.kFatal
%jsroot on

/home/yoren/.local/lib/python3.10/site-packages/nbformat/__init__.py:96: MissingIDFieldWarning: Cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


Welcome to JupyROOT 6.30/06


In [2]:
!rm output/DCA_fits/?/*.png

In [3]:
iOption0 = 0
iOption1 = [16,25]

In [4]:
isigma = 2
N_files = 1
N_centr = 5
is_sigmalised = 0
centralities = ["0-20%","20-40%","40-60%","60-80%","80-93%"]
centrality = [1,1,2,2,3,3,4,4,5,5]
layer_bins = [1,1,2,2,3,3,4,4,5,5,6,6]
colors=[1,2,4,root.kGreen+2,root.kMagenta,root.kOrange]
file_path="../VTXalignment/input/"
file_names="my-m_ee_Run14AuAu_107th_new_19943_1065runs.root"
hist_select_3D_names = ["DCA_2D_hist"]
N_hists = len(hist_select_3D_names)

In [5]:
hists_emc_read = []

for iFile in range(N_files):
    infile = root.TFile.Open(file_path+file_names, "read")
    hists_emc_read0 = []
    for icentr in range(N_centr):
        hist_select_3D0 = []
        for ihist in range(len(hist_select_3D_names)):
            hist_select_3D0.append(infile.Get(hist_select_3D_names[ihist]+f"_{icentr}"))
            hist_select_3D0[-1].SetDirectory(root.nullptr)
            #if icentr==4: hist_select_3D0[-1].Add(infile.Get(hist_select_3D_names[ihist]+f"_{icentr-1}"))
        hists_emc_read0.append(hist_select_3D0)
    hists_emc_read.append(hists_emc_read0)

infile.Close()

In [6]:
hist_for_pt = hists_emc_read[0][0][0]
for icentr in range(1,N_centr):
    hist_for_pt.Add(hists_emc_read[0][icentr][0])
pt_proj = hist_for_pt.ProjectionY("kek1",50,16,25)
c0 = root.TCanvas(f"c0",f"c0",720,360)
Format_Hist_total(pt_proj,"p_{T}","dN/dp_{T}",left=0.15, bottom=0.15, right=0, top=0,  Tsize=0.07,  Lsize=0.06,\
                      Mstyle=21,  Msize=1, Mcolor=4,  Lwidth=3,  Lcolor=4,  offset_x=1, offset_y=1, title="",  Malpha=1,  Lalpha=1)
pt_proj.GetXaxis().SetRange(3,49)
c0.SetLogy()
pt_proj.Draw("P")
myexp = root.TF1("myexp","([0]*0.1*([1]-1)*([1]-2))*(pow((sqrt(x*x+[3]*[3])+[2]-[3])/([2]),-[1]))/(2*3.14159)/([2]+[3]*([1]-2))/([2]+[3])",0.2,5)
myexp.SetParameter(3,0.135)
myexp.SetParameter(0,pt_proj.GetMaximum()*12)
myexp.SetParameter(1,15)
myexp.SetParameter(2,4.7)
pt_proj.Fit(myexp,"Q","",0.4,5)
c0.Draw()

In [7]:
pt = array( 'd' )
pt_boarders = array( 'd', [0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,1.3,1.7,2.2,3.0,4.9])
if is_sigmalised: pt_boarders = array( 'd', [0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,1.3,1.7,2.2,3.0,4.9])
pt_err = array( 'd' )
N_pt = len(pt_boarders) - 1
N_steps = 100
for i in range(N_pt):
    pt_step = pt_boarders[i]+(pt_boarders[i+1]-pt_boarders[i])/N_steps*(0+0.5)
    for istep in range(N_steps):
        pt_step = pt_boarders[i]+(pt_boarders[i+1]-pt_boarders[i])/N_steps*(istep+0.5)
        if myexp.Integral(pt_boarders[i],pt_step,1e-5) > myexp.Integral(pt_step,pt_boarders[i+1],1e-5):
            break
    pt.append( round(pt_step,4))
    pt_err.append((pt_boarders[i+1]-pt_boarders[i])/2)
print("pt = ", pt,"\npt_boarders = ",pt_boarders)

pt =  array('d', [0.3445, 0.4445, 0.5445, 0.6455, 0.7455, 0.8455, 0.9455, 1.1125, 1.442, 1.8725, 2.444, 3.3895]) 
pt_boarders =  array('d', [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.3, 1.7, 2.2, 3.0, 4.9])


In [8]:
def emc_fitter(InHist, seed=1, icentr=0, sigma=2, fits_mean=0, fits_sigma=0):
    N_gaus=1
    seed0 = seed
    seed=seed*N_pt*icentr*100

    hist_pt_bin = InHist.ProjectionY("kek")

    Mean_local =  array( 'd' )
    Sigma_local =  array( 'd' )
    Mean_local_err =  array( 'd' )
    Sigma_local_err =  array( 'd' )

    int_fg_local =  array( 'd' )
    int_bg_local =  array( 'd' )
 
    for p in range(N_pt):
        c1 = root.TCanvas(f"c1{seed+p}",f"c1{seed+p}",720,360)

        proj_local = InHist.ProjectionX(f"emc{seed+p}",hist_pt_bin.FindBin(pt_boarders[p]),hist_pt_bin.FindBin(pt_boarders[p+1])-1,iOption1[0],iOption1[1])
        #proj_local("CENTRLITY :", centrality[2*centr], "-",centrality[2*centr+1])
        #print("PT :", pt_boarders[p], "-",pt_boarders[p+1])
        #print("bins :", pt_boarders[p], "-",pt_boarders[p+1])
        proj_local.SetLineColor(1)

        gaus0 = root.TF1("gaus0"+proj_local.GetName(), "gaus", -150, 150)
        proj_local.Draw()
        proj_local.Fit(gaus0, "Q", "", -150, 150)
        gaus1 = root.TF1("gaus1"+proj_local.GetName(), "gaus", gaus0.GetParameter(1)-80,gaus0.GetParameter(1)+80)
        proj_local.Fit(gaus1, "Q", "", gaus0.GetParameter(1)-80,gaus0.GetParameter(1)+80)            
        gaus = root.TF1("gaus"+proj_local.GetName(), "gaus", gaus1.GetParameter(1)-gaus1.GetParameter(2)*2,gaus1.GetParameter(1)+gaus1.GetParameter(2)*2)
        gaus.SetParameters(gaus1.GetParameters())
        proj_local.Fit(gaus, "Q", "", gaus1.GetParameter(1)-gaus1.GetParameter(2)*1.09,gaus1.GetParameter(1)+gaus1.GetParameter(2)*1.09)


        gaus.Draw('same')

        Mean_local.append(gaus.GetParameter(1))
        Sigma_local.append(gaus.GetParameter(2))
        Mean_local_err.append(gaus.GetParError(1)+0.1*gaus.GetParameter(2))
        Sigma_local_err.append(gaus.GetParError(2)+0.1*gaus.GetParameter(2))
    
        int_fg_local.append(gaus.Integral(gaus.GetParameter(1)-sigma*gaus.GetParameter(2),gaus.GetParameter(1)+sigma*gaus.GetParameter(2)))
        int_bg_local.append(0)
            
        #m2_pos.Draw("same") m2_neg.Draw("same")
        m2_name=str(icentr)+"/dca_"+str(round(pt[p],2))

        if fits_mean==0:
            c1.SaveAs( "output/DCA_fits/" + m2_name + ".png" )
        else:
            c1.SaveAs( "output/DCA_fits/" + m2_name + "upd.png" )

    return [Mean_local, Mean_local_err, Sigma_local, Sigma_local_err, int_fg_local, int_bg_local]


In [9]:
def emc_centr_fitter(InHist, seed=0, sigma=2, fits_mean=0, fits_sigma=0):
    out_hists = []
    for icentr in range(N_centr):
        out_hists.append(emc_fitter(InHist[seed][icentr][iOption0], seed, icentr, sigma, fits_mean, fits_sigma))
    return out_hists

In [10]:
import multiprocess as mp

Ntr = N_files

pool = mp.Pool(Ntr)
output_array = pool.starmap(emc_centr_fitter, [(hists_emc_read, i, isigma) for i in range(Ntr)])
pool.close()

In [11]:
Means, Mean_errs, Sigmas, Sigma_errs, Int_FG, Int_BG = [], [], [], [], [], []
for ifile in range(Ntr):
    Means1, Mean_errs1, Sigmas1, Sigma_errs1, Int_FG1, Int_BG1 = [], [], [], [], [], []
    for icentr in range(N_centr):
            Means1.append(output_array[ifile][icentr][0])
            Mean_errs1.append(output_array[ifile][icentr][1])
            Sigmas1.append(output_array[ifile][icentr][2])
            Sigma_errs1.append(output_array[ifile][icentr][3])
            Int_FG1.append(output_array[ifile][icentr][4])
            Int_BG1.append(output_array[ifile][icentr][5])
    Means.append(Means1)
    Mean_errs.append(Mean_errs1)
    Sigmas.append(Sigmas1)
    Sigma_errs.append(Sigma_errs1)
    Int_FG.append(Int_FG1)
    Int_BG.append(Int_BG1)

In [12]:
types = ["emc_r_dphi_","emc_min_r_dphi_","emc_cnt_dphi_","emc_r_dz_","emc_min_r_dz_","emc_cnt_dz_"]
fit_functions = []
for ifile in range(N_files):
    fit_functions0 = []
    for icentr in range(N_centr):
            fit_functions0.append(root.TF1(f"fit_functions_sigma_{ifile}_{0}_"+centralities[icentr],"[0]+[1]*exp([2]*x)",0.2,5.0))
            fit_functions0[-1].SetParameter(0,80)
            fit_functions0[-1].SetParameter(1,200)
            fit_functions0[-1].SetParameter(2,-2)
            fit_functions0[-1].SetParLimits(2,-3.526,-0.023)
            fit_functions0[-1].SetParameter(3,-5)
            fit_functions0[-1].SetParLimits(3,-30.526,-0.023)
            fit_functions0[-1].SetLineColor(colors[icentr])
    fit_functions.append(fit_functions0)

In [13]:

print(Sigmas[0][0][0],Sigmas[0][3][0])
for ifile in range(N_files):
    h2=[]
    c3 = root.TCanvas(f"c3{ifile}",f"c3{ifile}",1500,900)
    c3.Divide(3,2)
    g_sigma = [[[]]*N_centr]*N_files
    min, max = 0, 400
    names = ["emc_r_dphi","emc_min_r_dphi","emc_cnt_dphi","emc_r_dz","emc_min_r_dz","emc_cnt_dz"]
    legends =[]
    for icentr in range(N_centr):
        c3.cd(icentr+1)
        h2.append(Format_Pad_old( 0.05, 5, min, max, "p_{T} (GeV/#it{c})", "Sigmas", 1.1, 1.1, 0.06, 0.05, "",0.15,0.15,0.,0.1))
        h2[-1].Draw()
        legends.append(root.TLegend(0.55,0.4,0.85,0.9,centralities[icentr]))
        legends[-1].SetFillColorAlpha(0,0)
        legends[-1].SetLineWidth(0)
        g_sigma[ifile][icentr].append(root.TGraphErrors(N_pt, pt, Sigmas[ifile][icentr],pt_err,Sigma_errs[ifile][icentr]))
        #g_sigma_inter[icentr][ilayer].append(root.TGraphErrors(N_pt))
        Format_Graph(g_sigma[ifile][icentr][-1],20+int(0/4),1, colors[icentr], 3, colors[icentr],1,1)
        #Format_Graph(g_sigma_inter[icentr][ilayer][-1], 21, 0, colors[ilayer], 3, colors[ilayer], 1, 1)
        g_sigma[ifile][icentr][-1].Fit(fit_functions[ifile][icentr],"Q","")
        #params_mean_sigma[syst][centr][0].append(all_fit_sigma_low[ii].GetParameters())
        g_sigma[ifile][icentr][-1].Draw('P')
        #g_sigma_inter[icentr][ilayer][-1].Draw('sameL')
        legends[-1].AddEntry(g_sigma[ifile][icentr][-1],f"layer {0}","p")
        #legends[-1].AddEntry(fit_functions[icentr][ilayer],f"fit","l")

        legends[-1].Draw()
    c3.Draw()
    c3.SaveAs(f"output/Sigmas/sigma_{ifile}{iOption0}{iOption0}.png")

145.58049098445426 142.23862155113454


python ERROR: cannot open image file "output/Sigmas/sigma_000.png" for writing. Please check permissions.


In [14]:
print(f"const float sigma_DCA[{N_centr}][3]=")
print("{")
for fits in fit_functions[0]:
        print("    {"+f"{fits.GetParameter(0):.1f}, {fits.GetParameter(1):.0f}, {fits.GetParameter(2):.2f}"+"},")
print("};")        

const float sigma_DCA[5][3]=
{
    {65.9, 193, -2.63},
    {66.0, 193, -2.67},
    {74.4, 215, -3.12},
    {71.1, 171, -2.66},
    {71.2, 124, -1.50},
};


In [15]:
fit_mean_functions = []
for ifile in range(N_files):
    fit_functions0 = []
    for icentr in range(N_centr):
        fit_functions0.append(root.TF1(f"fit_functions_mean_{ifile}_{icentr}_"+centralities[icentr],"[0]+[1]*exp([2]*x)",0.5,5.0))
        fit_functions0[-1].SetParameter(0,0.029)
        fit_functions0[-1].SetParameter(1,0.02)
        fit_functions0[-1].SetParameter(2,-2)
        fit_functions0[-1].SetParLimits(2,-3.126,-0.023)
        fit_functions0[-1].SetLineColor(colors[icentr])
    fit_mean_functions.append(fit_functions0)

In [16]:
for ifile in range(N_files):
    h2=[]
    c3 = root.TCanvas(f"c3",f"c3",1500,900)
    c3.Divide(3,2)
    g_mean = [[[]]*N_centr]*N_files
    min, max = -500, 500
    if iOption1==1: max, min = -1, 1
    names = ["emc_r_dphi","emc_min_r_dphi","emc_cnt_dphi","emc_r_dz","emc_min_r_dz","emc_cnt_dz"]
    legends =[]
    for icentr in range(N_centr):
        c3.cd(icentr+1)
        h2.append(Format_Pad_old( 0.05, 5, min, max, "p_{T} (GeV/#it{c})", "Means", 1.1, 1.1, 0.06, 0.05, "",0.15,0.15,0.,0.1))
        h2[-1].Draw()
        legends.append(root.TLegend(0.55,0.4,0.85,0.9,centralities[icentr]))
        legends[-1].SetFillColorAlpha(0,0)
        legends[-1].SetLineWidth(0)

        g_mean[ifile][icentr].append(root.TGraphErrors(N_pt, pt, Means[ifile][icentr],pt_err,Mean_errs[ifile][icentr]))
        #g_sigma_inter[icentr][ilayer].append(root.TGraphErrors(N_pt))
        Format_Graph(g_mean[ifile][icentr][-1],20+int(icentr/4),1, colors[icentr], 3, colors[icentr],1,1)
        #Format_Graph(g_sigma_inter[icentr][ilayer][-1], 21, 0, colors[ilayer], 3, colors[ilayer], 1, 1)
        g_mean[ifile][icentr][-1].Fit(fit_mean_functions[ifile][icentr],"Q","")
        #params_mean_sigma[syst][centr][0].append(all_fit_sigma_low[ii].GetParameters())
        g_mean[ifile][icentr][-1].Draw('P')
        #g_sigma_inter[icentr][ilayer][-1].Draw('sameL')
        legends[-1].AddEntry(g_mean[ifile][icentr][-1],f"layer {icentr}","p")
        #legends[-1].AddEntry(fit_functions[icentr][ilayer],f"fit","l")

        legends[-1].Draw()
    c3.Draw()
    c3.SaveAs(f"output/Sigmas/mean_{ifile}{iOption0}{iOption1}.png")

python ERROR: cannot open image file "output/Sigmas/mean_00[16, 25].png" for writing. Please check permissions.


In [17]:
print(f"const float params[{N_centr}][3]=")
print("{")
for fits in fit_mean_functions[0]:
    print("    {"+f"{round(fits.GetParameter(0),3)}, {round(fits.GetParameter(1),3)}, {round(fits.GetParameter(2),3)}"+"},")
print("};")        

const float params[5][3]=
{
    {-5.545, -74.635, -3.126},
    {-4.731, -72.353, -3.126},
    {-6.374, -57.844, -3.126},
    {-3.829, -46.643, -3.126},
    {26.991, -53.596, -0.187},
};
